In [1]:
import numpy as np
import torch
import torch.nn as nn
import csv
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time

In [2]:
from setup_dataset import *
from setup_model import *
from setup_model_types import *

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
data = chunk_dataset(match_list=[1],
                     start_frame = 1,
                     end_frame = 1030
                    )

Successfully loaded NPZ.
Do you want to use GPU (y/n)n
Selected CPU


C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:133: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train = np.array(X_train)
C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:134: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  Y_train = np.array(Y_train)


In [5]:
len(data.data)

124

In [6]:
data.padSequence()

In [7]:
original_shape = data.data.shape
original_shape

torch.Size([124, 30, 100, 100])

In [8]:
data.data = data.data.reshape(-1, 1, 100, 100)

In [9]:
data.data.shape, data.targets.shape, data.seq_len.shape

(torch.Size([3720, 1, 100, 100]), torch.Size([124, 30, 9]), torch.Size([124]))

In [10]:
data.seq_len

tensor([29., 16., 10.,  7., 18., 26., 11., 20.,  5., 14.,  3.,  7., 19., 30.,
        11.,  9.,  7., 10.,  7., 20.,  2., 11.,  4.,  3.,  4.,  5.,  7.,  3.,
         3.,  8.,  4.,  2., 19.,  6., 14.,  3.,  5.,  4.,  4.,  3.,  9., 12.,
         6., 24.,  3., 18.,  4., 18.,  4., 13.,  2.,  4.,  3., 12.,  3., 10.,
        14.,  6., 12.,  6.,  5.,  3.,  4.,  7.,  3.,  6.,  3.,  3.,  8.,  1.,
         3.,  9., 10.,  4., 17., 10., 13.,  8.,  4.,  7., 14.,  6.,  2.,  9.,
         4.,  6.,  9.,  1., 15.,  3., 16.,  1.,  9.,  6.,  4., 14., 11., 16.,
         4.,  3., 11.,  6.,  1.,  2.,  6.,  2., 19.,  2.,  5.,  8.,  4.,  3.,
         6., 11., 19.,  3., 19.,  7.,  8.,  8.,  3.,  5.,  2.,  1.])

In [11]:
device = data.device
input_size = data.data.shape[2]
output_size = len(data.available_targets)
match_list = data.match_list
start_frame = data.start_frame
end_frame = data.end_frame

In [12]:
data.seq_len.shape

torch.Size([124])

In [13]:
class CNNLSTMModel1(nn.Module):
    def __init__(self, device, input_size, output_size, hidden_dim, n_layers):
        super(CNNLSTMModel1, self).__init__()
        
        self.device = device
        self.lstm_input_size = input_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(3, 3, (10,10))
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(num_features=3)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

        # output = (input - filter + 1) / stride
        # convolução 1: (100 - 3 + 1) / 1 = 98x98
        # pooling 1: 49x49
        # convolução 2: (45 - 10 + 1) / 1 = 36x36
        # pooling 2: 18x18
        # 18 * 18 * 3
        self.init_hidden(49*49*3)
        self.lstm = nn.LSTM(49*49*3, hidden_dim, n_layers, batch_first=True)  
        self.fc = nn.Linear(hidden_dim, output_size)
        self.out = nn.Softmax()
    
    def forward(self, x):
        
        hidden = self.init_hidden(49*49*3)

        x = self.pool(self.bnorm(self.activation(self.conv1(x))))
        # x = self.pool(self.bnorm(self.activation(self.conv2(x))))

        x = x.reshape(original_shape[0], original_shape[1], -1)
        
        x = pack_padded_sequence(x, data.seq_len, batch_first=True, enforce_sorted=False)
        
        pad_embed_pack_lstm = self.lstm(x, hidden)
        pad_embed_pack_lstm_pad = pad_packed_sequence(pad_embed_pack_lstm[0], batch_first=True)
        
        outs, _ = pad_embed_pack_lstm_pad
        
        out = outs.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = self.out(out)
        
        return out
        
    def init_hidden(self, input_size):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(1, input_size, self.hidden_dim)
        hidden_b = torch.randn(1, input_size, self.hidden_dim)

        if self.device.type == 'cuda':
            hidden_a = hidden_a.cuda()
            hidden_b = hidden_b.cuda()

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

In [14]:
model_structure = ModelStructure(device, input_size, output_size, match_list, start_frame, end_frame)

Number of hidden neurons: 200
Number of epochs: 5000
Choose type of RNN model:
1 - Simple RNN
2 - LSTM
3 - CNN
type: 3
write a observations without space and punctuations:teste
models/CNN_teste_m1_f1to1030_epoch5000_H200
ATTENTION! folder not created. Training informations will overwrite the existing one


In [15]:
model = CNNLSTMModel1(device=model_structure.device, 
                     input_size=model_structure.data_size, 
                     output_size=model_structure.output_size, 
                     hidden_dim=model_structure.hidden_neurons, 
                     n_layers=1)

In [16]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [17]:
mymodel = model
X_train = data.data 
Y_train = data.targets

In [18]:
def get_acc(predicted, target):
    
    predicted = torch.argmax(predicted, axis=1)
    target = torch.argmax(target, axis=1)

    acc = np.mean(predicted == target)
    
    return float(acc)

In [19]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])

In [20]:
X_train.shape, Y_train.shape

(torch.Size([3720, 1, 100, 100]), torch.Size([124, 30, 9]))

In [ ]:
start_time_processing = time.time()

loss_file = open(model_structure.path + '/' + "loss_file.txt", "w")
first_time = True

best_loss = 1
first_epoch = True

for epoch in range(1, model_structure.n_epochs + 1):

    mymodel.train()

    optimizer.zero_grad()
    output = mymodel(X_train)
    loss = criterion(output, Y_train.view(-1,data.target_size[2]).float())
    loss.backward()
    optimizer.step()
        
    if epoch%10 == 0:

        train_loss_arr = np.append(train_loss_arr, loss.item())
        train_acc_arr  = np.append(train_acc_arr, model_structure.get_acc(output, Y_train.reshape(-1, len(data.available_targets))))
        
        loss_file.write("Epoch: {}/{}-------------------------------------------\n".format(epoch, model_structure.n_epochs))
        loss_file.write("Train -> Loss: {:.15f} Acc: {:.15f}\n".format(train_loss_arr[-1], train_acc_arr[-1]))
            
        print("Epoch: {}/{}-------------------------------------------".format(epoch, model_structure.n_epochs))
        print("Train -> Loss: {:.15f} Acc: {:.15f}".format(train_loss_arr[-1], train_acc_arr[-1]))
        
        if train_loss_arr[-1] < best_loss:
            state = { 'epoch': epoch + 1, 'state_dict': mymodel.state_dict(),
                      'optimizer': optimizer.state_dict(), 'losslogger': loss.item(), }
            torch.save(state, model_structure.path + '/' + model_structure.name)
            best_loss = loss.item()
        else:
            print("model not saved")
            
loss_file.write("--- %s seconds ---" % (time.time() - start_time_processing))
loss_file.close()
np.savez(model_structure.path + '/' + "train_loss_arr", train_loss_arr)
print("--- %s seconds ---" % (time.time() - start_time_processing))

C:\Users\alece\miniconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 10/5000-------------------------------------------
Train -> Loss: 0.019208239391446 Acc: 0.222849458456039
Epoch: 20/5000-------------------------------------------
Train -> Loss: 0.013902365230024 Acc: 0.252150535583496
Epoch: 30/5000-------------------------------------------
Train -> Loss: 0.011649702675641 Acc: 0.264247298240662
Epoch: 40/5000-------------------------------------------
Train -> Loss: 0.010395962744951 Acc: 0.269623667001724
Epoch: 50/5000-------------------------------------------
Train -> Loss: 0.009752138517797 Acc: 0.271774202585220


In [ ]:
output[0].shape

In [ ]:
output[1].shape

In [ ]:
output[2].shape

In [ ]:
data.seq_len

In [ ]:
data.data.shape

In [ ]:
data.targets.shape

In [ ]:
output.shape

In [ ]:
Y_train.view(-1,data.target_size[2]).shape

In [ ]:
data.seq_len.shape

In [ ]:
data.padSequence()

In [ ]:
data.packPaddedSequence()

In [ ]:
data.data[0].shape